In [15]:
import numpy as np
import pandas as pd
import sqlite3
import plotly.express as px
import matplotlib.pyplot as plt

In [16]:
# Step 2: Load the datasets
conn = sqlite3.connect('traffic_violation.sqlite')
parking_df = pd.read_sql_query("SELECT * FROM parking_violations", conn)
street_df = pd.read_sql_query("SELECT * FROM street_directory", conn)


In [39]:
print(len(parking_df['Crime_location'].unique()))
print(len(street_df['Street_Name'].unique()))




17141
2246


In [5]:
import pandas as pd

# Assuming you have the data frames parking_df and street_df

# Extract unique values from parking_df['Crime_location']
parking_unique_values = set(parking_df['Crime_Str_No'].unique())

# Extract unique values from street_df['Street_Name']
street_unique_values = set(street_df['Street_Number'].unique())

# Find the unique values that are in both sets
common_values = parking_unique_values & street_unique_values

# Print the common values
print(len(common_values))


704


In [12]:
# Step 3: Merge the datasets
merged_df = pd.merge(parking_df, street_df, left_on='Crime_Str_No', right_on='Street_Number', how='inner')
grouped_df = merged_df.groupby(['Crime_Str_No', 'Street_Number', 'Street_Name', 'latitude', 'longitude']).size().reset_index(name='Total Violations')
print(len(grouped_df))

704


In [80]:
print(merged_df)

       Crime_Str_No                                     Crime_location  \
0              1606                         Bonn, Lyngsbergstra√üe 108   
1              1606              Bonn, Siemensstra√üe Parkplatz Sirius   
2              1606         Bonn, Wilhelmstra√üe gegen√ºber Hnr. 40/42   
3              1606                  Bonn, F√ºrstenstra√üe Ecke Am Hof   
4              1606  Bonn, Obere Wilhelmstra√üe Ecke Beueler Bahnho...   
...             ...                                                ...   
47618          2340                           Friesdorfer Str. 248-250   
47619          2340                               Friesdorfer Str. 250   
47620          2340                               Friesdorfer Str. 252   
47621          2340                      Bonn, Villichgasse vor Hnr. 2   
47622          2346                        Bonn, Villichgasse ohne Nr.   

       Crime_Violation_No  Total_Fine Vehicle_Description  Street_Number  \
0                  112293          

In [81]:
# Step 4: Create a scatter plot
fig = px.scatter(grouped_df, x='Street_Number', y='Street_Name', size='Total Violations', color='Total Violations')



In [13]:
# Step 5: Display the plot
fig.show()  # Use this for plotly



NameError: name 'fig' is not defined

In [83]:
print(grouped_df.columns)

Index(['Crime_Str_No', 'Street_Number', 'Street_Name', 'latitude', 'longitude',
       'Total Violations'],
      dtype='object')


In [84]:
print(len(grouped_df))

704


In [88]:
import folium
import pandas as pd

# Sample data
data = grouped_df

# Convert data to DataFrame
df = pd.DataFrame(data)

# Specify the latitude and longitude to center the map
center_latitude = df['latitude'][0]
center_longitude = df['longitude'][0]

# Create a folium map and set its center location
m = folium.Map(location=[center_latitude, center_longitude], zoom_start=13)

# Add circles to the map
for index, row in df.iterrows():
    street_name = row['Street_Name']
    total_violations = row['Total Violations']
    
    # Create a pop-up message with street_name and Total Violations
    popup_message = f"Street Name: {street_name}<br>Total Violations: {total_violations}"
    
    # Add circle marker with pop-up to the map
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['Total Violations'] * 0.25,  # Adjust the size scaling factor as needed
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        popup=popup_message
    ).add_to(m)

# Save the map to an HTML file
m.save('map.html')